In [28]:
from sqlalchemy import create_engine,text
import sqlalchemy
from urllib.parse import quote
import pandas as pd
from logzero import logger
from SmartApi.smartConnect import SmartConnect
import pyotp
import json
import re
import uuid
from uuid import UUID


In [29]:
stock_names = sorted([
    "AARTIIND", "ABB", "ABBOTINDIA", "ABCAPITAL", "ABFRL", "ACC", "ADANIENT", "ADANIPORTS", 
    "ALKEM", "AMBUJACEM", "APOLLOHOSP", "APOLLOTYRE", "ASHOKLEY", "ASIANPAINT", "ASTRAL", 
    "ATUL", "AUBANK", "AUROPHARMA", "AXISBANK", "BAJAJ-AUTO", "BAJAJFINSV", "BAJFINANCE", 
    "BALKRISIND", "BALRAMCHIN", "BANDHANBNK", "BANKBARODA", "BATAINDIA", "BEL", "BERGEPAINT", 
    "BHARATFORG", "BHARTIARTL", "BHEL", "BIOCON", "BOSCHLTD", "BPCL", "BRITANNIA", "BSOFT", 
    "CANBK", "CANFINHOME", "CHAMBLFERT", "CHOLAFIN", "CIPLA", "COALINDIA", "COFORGE", "COLPAL", 
    "CONCOR", "COROMANDEL", "CROMPTON", "CUB", "CUMMINSIND", "DABUR", "DALBHARAT", "DEEPAKNTR", 
    "DIVISLAB", "DIXON", "DLF", "DRREDDY", "EICHERMOT", "ESCORTS", "EXIDEIND", "FEDERALBNK", 
    "GAIL", "GLENMARK", "GMRINFRA", "GNFC", "GODREJCP", "GODREJPROP", "GRANULES", "GRASIM", 
    "GUJGASLTD", "HAL", "HAVELLS", "HCLTECH", "HDFCAMC", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", 
    "HINDALCO", "HINDCOPPER", "HINDPETRO", "HINDUNILVR", "ICICIBANK", "ICICIGI", "ICICIPRULI", 
    "IDEA", "IDFC", "IDFCFIRSTB", "IEX", "IGL", "INDHOTEL", "INDIACEM", "INDIAMART", "INDIGO", 
    "INDUSINDBK", "INDUSTOWER", "INFY", "IOC", "IPCALAB", "IRCTC", "ITC", "JINDALSTEL", 
    "JKCEMENT", "JSWSTEEL", "JUBLFOOD", "KOTAKBANK", "LALPATHLAB", "LAURUSLABS", "LICHSGFIN", 
    "LT", "LTF", "LTIM", "LTTS", "LUPIN", "M&M", "M&MFIN", "MANAPPURAM", "MARICO", "MARUTI", 
    "MCX", "METROPOLIS", "MFSL", "MGL", "MOTHERSON", "MPHASIS", "MRF", 
    "MUTHOOTFIN", "NATIONALUM", "NAUKRI", "NAVINFLUOR", "NESTLEIND", "NMDC", "NTPC", 
    "OBEROIRLTY", "OFSS", "ONGC", "PAGEIND", "PEL", "PERSISTENT", "PETRONET", "PFC", 
    "PIDILITIND", "PIIND", "PNB", "POLYCAB", "POWERGRID", "PVRINOX", "RAMCOCEM", "RBLBANK", 
    "RECLTD", "RELIANCE", "SAIL", "SBICARD", "SBILIFE", "SBIN", "SHREECEM", "SHRIRAMFIN", 
    "SIEMENS", "SRF", "SUNPHARMA", "SUNTV", "SYNGENE", "TATACHEM", "TATACOMM", "TATACONSUM", 
    "TATAMOTORS", "TATAPOWER", "TATASTEEL", "TCS", "TECHM", "TITAN", "TORNTPHARM", "TRENT", 
    "TVSMOTOR", "UBL", "ULTRACEMCO", "UNITDSPR", "UPL", "VEDL", "VOLTAS", "WIPRO", 
    "ZYDUSLIFE"], key=len, reverse=True)
NAMESPACE_STOCK = UUID('233c16a9-0a91-4c9d-adda-8a496c63a1a3')

expiry_dates = ['2024-07-25']
api_key = 'BP42pHUk'
username = 'R60865380'
pwd = '6713'
smartApi = SmartConnect(api_key)

engine = create_engine(
        "postgresql+psycopg2://sd_admin:%s@192.168.1.72:5430/stock-dumps"
        % quote("sdadmin@postgres")
    )

[I 240730 01:49:37 smartConnect:121] in pool


In [33]:
try:
    token = "ARA7CTUQBDWODFCWKVSAXTLR4U"
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)
def categorize_tradingsymbol(row, stock_names):
    for name in stock_names:
        if re.match(f"^{name}", row['tradingsymbol']):
            return name
    return None
searchScripData_df = pd.DataFrame({})
if data['status'] == False:
    logger.error(data)
else:
    # logger.info(f"data: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    feedToken = smartApi.getfeedToken()
    # logger.info(f"Feed-Token :{feedToken}")
    res = smartApi.getProfile(refreshToken)
    # logger.info(f"Get Profile: {res}")
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']
    #search for symbol
    exchange = "NFO"
    searchscrip = "ABBOTINDIA"
    searchScripData = smartApi.searchScrip(exchange, searchscrip)
    df = pd.DataFrame(
        searchScripData["data"], columns=["exchange", "tradingsymbol", "symboltoken"]
    )
    df['stock_name'] = df.apply(categorize_tradingsymbol, stock_names=stock_names, axis=1)
    searchScripData_df = df[~df.duplicated(['symboltoken']) & df["tradingsymbol"].str.contains("AUG")]
    print(df[~df.duplicated(['symboltoken']) & df["tradingsymbol"].str.contains("AUG")])

[I 240730 02:37:05 smartConnect:485] Search successful. Found 420 trading symbols for the given query:
    1. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2416500CE, symboltoken: 63409
    2. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2416500PE, symboltoken: 63410
    3. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2417000CE, symboltoken: 90405
    4. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2417000PE, symboltoken: 90406
    5. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2417500CE, symboltoken: 63411
    6. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2417500PE, symboltoken: 63412
    7. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2418000CE, symboltoken: 90407
    8. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2418000PE, symboltoken: 90408
    9. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2418500CE, symboltoken: 90413
    10. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2418500PE, symboltoken: 90414
    11. exchange: NFO, tradingsymbol: ABBOTINDIA26SEP2419000CE, symboltoken:

    exchange             tradingsymbol symboltoken  stock_name
166      NFO  ABBOTINDIA29AUG2416500CE       90268  ABBOTINDIA
167      NFO  ABBOTINDIA29AUG2416500PE       90269  ABBOTINDIA
168      NFO  ABBOTINDIA29AUG2417000CE       76996  ABBOTINDIA
169      NFO  ABBOTINDIA29AUG2417000PE       76997  ABBOTINDIA
170      NFO  ABBOTINDIA29AUG2417500CE       76998  ABBOTINDIA
..       ...                       ...         ...         ...
323      NFO  ABBOTINDIA29AUG2439500PE       90400  ABBOTINDIA
324      NFO  ABBOTINDIA29AUG2440000CE       52084  ABBOTINDIA
325      NFO  ABBOTINDIA29AUG2440000PE       52087  ABBOTINDIA
326      NFO  ABBOTINDIA29AUG2440500CE       54111  ABBOTINDIA
327      NFO  ABBOTINDIA29AUG2440500PE       54122  ABBOTINDIA

[162 rows x 4 columns]


In [16]:
def query_to_dataframe(query, connection):
    result = connection.execute(text(query))
    rows = result.fetchall()
    columns = result.keys()
    return pd.DataFrame(rows, columns=columns)

# Connect to the database
with engine.connect() as connection:
    # Query to select all from the 'stock' table
    stock_df = query_to_dataframe("SELECT * FROM options.stock", connection)


In [12]:
searchScripData_df
# create ticker df 
# symbol_token stock_id expiry_date strike_price option_type

ticker_df = pd.DataFrame(
    columns=["symbol_token", "stock_id", "expiry_date", "strike_price", "option_type"]
)
ticker_df['symbol_token'] = searchScripData_df['symboltoken']
ticker_df['stock_id'] = 
print(ticker_df)

Empty DataFrame
Columns: [symbol_token, stock_id, expiry_date, strike_price, option_type]
Index: []


In [30]:
# create stocks df
# name nifty_fifty_index
df = pd.DataFrame(
        {'name':stock_names, 'nifty_fifty_index':0}
    )
df['id'] = [uuid.uuid5(NAMESPACE_STOCK,s) for s in stock_names]
print(df)

           name  nifty_fifty_index                                    id
0    ABBOTINDIA                  0  13512523-55e9-57e4-9e0c-22fd88120b84
1    ADANIPORTS                  0  6396eba1-a7df-5169-8ace-1611c5d12a22
2    APOLLOHOSP                  0  f22055cc-136c-5f1c-9117-30ecf02ead7a
3    APOLLOTYRE                  0  b9fcab46-8164-5a65-b4ef-5777fe4b7f0e
4    ASIANPAINT                  0  05a764e9-f3d3-5acd-8372-f0f9453a90e2
..          ...                ...                                   ...
176         SRF                  0  32a1bca4-0d0b-5b7e-8fa1-5c8985bd552c
177         TCS                  0  ed5e8f3c-cd82-587a-a598-c4f6eae04380
178         UBL                  0  66aedc2e-97d6-5d9b-9d15-66d348f2a110
179         UPL                  0  98160b98-f2ad-5306-8c05-9b9d1b734fb9
180          LT                  0  754217d7-4b2a-5af8-9c72-31a14a28fbc3

[181 rows x 3 columns]


In [31]:
# insert into stock table
df.to_sql("stock", schema="options", if_exists="append", con=engine, index=False)

181

In [12]:
# read expiry, stock to a df
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM options.stock"))
    rows = result.fetchall()
    columns = result.keys()
df = pd.DataFrame(rows, columns=columns)
print(df)

     stock_id        name  nifty_fifty_index
0           1  ABBOTINDIA                  0
1           2  ADANIPORTS                  0
2           3  APOLLOHOSP                  0
3           4  APOLLOTYRE                  0
4           5  ASIANPAINT                  0
..        ...         ...                ...
176       177         SRF                  0
177       178         TCS                  0
178       179         UBL                  0
179       180         UPL                  0
180       181          LT                  0

[181 rows x 3 columns]
